In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

# iteration

In [4]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 20
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)
        
# supervised attention factor
delta_list = [0.5, 1, 1.5, 2, 2.5]
att_list = ['linear', 'exp']
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

for iteration in range(1,11):
    for att_type in att_list:
        for delta in delta_list:
            print(f"{iteration}th iteration")
            print(f"att_type: {att_type}")
            print(f"delta: {delta}")
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
print('end opitmization')

1th iteration
att_type: linear
delta: 0.5
Epoch 1/10000
563/563 - 11s - loss: 0.2280 - val_loss: 0.1835 - 11s/epoch - 20ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.1593 - val_loss: 0.1819 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 6s - loss: 0.1485 - val_loss: 0.1574 - 6s/epoch - 11ms/step
Epoch 4/10000
563/563 - 6s - loss: 0.1399 - val_loss: 0.1534 - 6s/epoch - 10ms/step
Epoch 5/10000
563/563 - 6s - loss: 0.1316 - val_loss: 0.1411 - 6s/epoch - 10ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1251 - val_loss: 0.1371 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1186 - val_loss: 0.1341 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1134 - val_loss: 0.1302 - 5s/epoch - 9ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1084 - val_loss: 0.1200 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1042 - val_loss: 0.1241 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.0990 - val_loss: 0.1101 - 5s/epoch - 9ms/step
Epoch 12/10000
563/563 - 6s - loss: 0

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.846615,0.907208,0.938124,0.897316,7.727636,5.356199,3.820904,5.634913
1,0.796534,0.850468,0.908163,0.851721,8.740353,6.801561,4.656307,6.73274
2,0.707208,0.799141,0.870673,0.792341,10.028142,7.885955,5.527188,7.813761
3,0.610921,0.757385,0.840415,0.73624,11.173961,8.670024,6.141176,8.66172
4,0.515775,0.719383,0.818827,0.684662,12.266004,9.328495,6.544863,9.379787
5,0.421917,0.686782,0.802564,0.637088,13.356922,9.859945,6.833493,10.016787
6,0.332165,0.657959,0.789875,0.593333,14.07488,10.305971,7.051096,10.477316
7,0.251936,0.632061,0.778002,0.553999,14.611558,10.691415,7.248981,10.850651
8,0.182839,0.608312,0.761364,0.517505,15.045828,11.032973,7.517062,11.198621


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_linear_0.5_1.csv


1th iteration
att_type: linear
delta: 1
Epoch 1/10000
563/563 - 9s - loss: 0.3080 - val_loss: 0.2517 - 9s/epoch - 15ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.2222 - val_loss: 0.2325 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2056 - val_loss: 0.2195 - 5s/epoch - 8ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.1944 - val_loss: 0.2098 - 5s/epoch - 8ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.1839 - val_loss: 0.2024 - 5s/epoch - 8ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1758 - val_loss: 0.1919 - 5s/epoch - 8ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1675 - val_loss: 0.1895 - 5s/epoch - 8ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1610 - val_loss: 0.1779 - 5s/epoch - 8ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1561 - val_loss: 0.1716 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1481 - val_loss: 0.1687 - 5s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1424 - 

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.89091,0.883243,0.937504,0.903886,6.516988,6.008184,3.839989,5.455054
1,0.817476,0.843311,0.906571,0.855786,8.278315,6.962429,4.696508,6.64575
2,0.728403,0.790658,0.854301,0.791121,9.658361,8.05076,5.866621,7.858581
3,0.641329,0.761546,0.80798,0.736951,10.728439,8.595364,6.736417,8.68674
4,0.559351,0.736445,0.770419,0.688738,11.701074,9.040461,7.367533,9.369689
5,0.488053,0.713878,0.736677,0.646203,12.569666,9.423814,7.891771,9.96175
6,0.426952,0.692381,0.715451,0.611595,13.037838,9.773632,8.205335,10.338935
7,0.375572,0.672568,0.70534,0.584493,13.3496,10.085735,8.351481,10.595605
8,0.334246,0.652771,0.697157,0.561391,13.580604,10.387974,8.468161,10.812246


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_linear_1_1.csv


1th iteration
att_type: linear
delta: 1.5
Epoch 1/10000
563/563 - 9s - loss: 0.4277 - val_loss: 0.3429 - 9s/epoch - 16ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.2962 - val_loss: 0.3062 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2737 - val_loss: 0.3015 - 5s/epoch - 8ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.2565 - val_loss: 0.2752 - 5s/epoch - 8ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.2414 - val_loss: 0.2691 - 5s/epoch - 8ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2301 - val_loss: 0.2515 - 5s/epoch - 8ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.2206 - val_loss: 0.2402 - 5s/epoch - 8ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.2084 - val_loss: 0.2275 - 5s/epoch - 8ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1992 - val_loss: 0.2211 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1916 - val_loss: 0.2117 - 5s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1832 - 

...

csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_0.5_4.csv


4th iteration
att_type: exp
delta: 1
Epoch 1/10000
563/563 - 9s - loss: 0.2734 - val_loss: 0.2265 - 9s/epoch - 16ms/step
Epoch 2/10000
563/563 - 6s - loss: 0.2016 - val_loss: 0.2087 - 6s/epoch - 11ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.1891 - val_loss: 0.1958 - 5s/epoch - 10ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.1778 - val_loss: 0.1898 - 5s/epoch - 10ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.1707 - val_loss: 0.1814 - 5s/epoch - 10ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1621 - val_loss: 0.1823 - 5s/epoch - 10ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1553 - val_loss: 0.1643 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 6s - loss: 0.1512 - val_loss: 0.1603 - 6s/epoch - 10ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1394 - val_loss: 0.1575 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1344 - val_loss: 0.1489 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1281 - 

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.874374,0.917467,0.951198,0.914346,6.993491,5.05144,3.393328,5.146086
1,0.826896,0.851488,0.903653,0.860679,8.061868,6.778317,4.769281,6.536489
2,0.754067,0.789481,0.835195,0.792915,9.190707,8.073352,6.239428,7.834495
3,0.671669,0.744802,0.774908,0.73046,10.264647,8.892023,7.293493,8.816721
4,0.588695,0.718016,0.747619,0.684777,11.304762,9.351199,7.724717,9.460226
5,0.512349,0.702799,0.738513,0.65122,12.267781,9.60454,7.864213,9.912178
6,0.446006,0.691508,0.727908,0.621807,12.819248,9.787498,8.023726,10.210157
7,0.389607,0.68035,0.708693,0.592883,13.198724,9.965158,8.303829,10.489237
8,0.341372,0.667629,0.676034,0.561678,13.507727,10.163294,8.758506,10.809842


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_1_4.csv


4th iteration
att_type: exp
delta: 1.5
Epoch 1/10000
563/563 - 9s - loss: 0.3735 - val_loss: 0.2908 - 9s/epoch - 17ms/step
Epoch 2/10000
563/563 - 6s - loss: 0.2534 - val_loss: 0.2690 - 6s/epoch - 10ms/step
Epoch 3/10000
563/563 - 6s - loss: 0.2368 - val_loss: 0.2438 - 6s/epoch - 10ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.2231 - val_loss: 0.2343 - 5s/epoch - 10ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.2128 - val_loss: 0.2302 - 5s/epoch - 10ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2027 - val_loss: 0.2161 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1941 - val_loss: 0.2117 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1848 - val_loss: 0.2016 - 5s/epoch - 10ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1797 - val_loss: 0.2038 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1685 - val_loss: 0.1926 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1617 - v

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.879727,0.922614,0.931727,0.911356,6.842876,4.891393,4.013571,5.24928
1,0.796197,0.861937,0.887234,0.848456,8.747588,6.535515,5.159663,6.814256
2,0.709444,0.810274,0.835169,0.784962,9.989777,7.664286,6.239931,7.964665
3,0.622598,0.76983,0.795225,0.729218,11.005008,8.444738,6.956549,8.802098
4,0.540664,0.739686,0.761823,0.680724,11.946607,8.984701,7.504192,9.4785
5,0.461215,0.715275,0.733286,0.636592,12.894934,9.400773,7.942415,10.079374
6,0.388923,0.694842,0.713534,0.5991,13.463505,9.734469,8.232927,10.476967
7,0.327274,0.67602,0.701979,0.568424,13.856265,10.032435,8.398965,10.762555
8,0.278402,0.658407,0.688679,0.541829,14.138711,10.303316,8.585872,11.0093


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_1.5_4.csv


4th iteration
att_type: exp
delta: 2
Epoch 1/10000
563/563 - 9s - loss: 0.4465 - val_loss: 0.3530 - 9s/epoch - 16ms/step
Epoch 2/10000
563/563 - 7s - loss: 0.3114 - val_loss: 0.3272 - 7s/epoch - 12ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2888 - val_loss: 0.3170 - 5s/epoch - 9ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.2730 - val_loss: 0.2901 - 5s/epoch - 9ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.2562 - val_loss: 0.2764 - 5s/epoch - 10ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2437 - val_loss: 0.2629 - 5s/epoch - 10ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.2284 - val_loss: 0.2528 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.2186 - val_loss: 0.2527 - 5s/epoch - 10ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.2097 - val_loss: 0.2262 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 6s - loss: 0.1976 - val_loss: 0.2246 - 6s/epoch - 10ms/step
Epoch 11/10000
563/563 - 7s - loss: 0.1893 - v